In [1]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/infer_data/train.parquet')
df.ts = (df.ts//1000).astype('int32')

mx = df.ts.max()
df = df.loc[df.ts >= mx-60*60*24*21]

df['hour'] = df.ts % (60*60*24)

df = df.loc[ (df.hour >= 60*60*14)|(df.hour < 60*60*2) ]

df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(91951194, 6)


,session,aid,ts,type,hour,day
76,0,1225559,1660157753,0,68153,586553
77,0,240346,1660157783,0,68183,586583
78,0,1582117,1660244973,0,68973,68973
80,0,1624436,1660427603,0,78803,251603
81,0,1157411,1660427638,0,78838,251638


In [3]:
df = df.sort_values(['session','ts'])
df['prev'] = df.groupby('session').aid.diff(1)
df['next'] = df.groupby('session').aid.diff(-1)

df['order'] = (df.type==2).astype('int8')
df['cart'] = (df.type==1).astype('int8')

In [4]:
df.head()

,session,aid,ts,type,hour,day,prev,next,order,cart
76,0,1225559,1660157753,0,68153,586553,<NA>,985213,0,0
77,0,240346,1660157783,0,68183,586583,-985213,-1341771,0,0
78,0,1582117,1660244973,0,68973,68973,1341771,-42319,0,0
80,0,1624436,1660427603,0,78803,251603,42319,467025,0,0
81,0,1157411,1660427638,0,78838,251638,-467025,799106,0,0


In [5]:
df['n'] = df.groupby(['session','aid','type']).aid.transform('count')
tmp = df.loc[df['type']==2].drop_duplicates(['session','aid']).groupby('aid').n.mean()
tmp2 = df.loc[df['type']==1].drop_duplicates(['session','aid']).groupby('aid').n.mean()

In [6]:
tmp.head()

aid
1199967    <NA>
136534     <NA>
1536528    <NA>
1745318    <NA>
1330002    <NA>
Name: n, dtype: float64

In [7]:
tmp.max(), tmp2.max()

(462.0, 462.0)

In [8]:
item_features3 = df.groupby('aid').agg({'prev':'nunique','next':'nunique','order':'sum','cart':'sum'})
item_features3 = item_features3.merge(tmp,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','buy_count3']
item_features3 = item_features3.merge(tmp2,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','order_repeat3','cart_repeat3']
item_features3 = item_features3.fillna(-1)

i32 = ['prev3','next3','orders3','carts3']
for c in i32: item_features3[c] = item_features3[c].astype('int32')
    
f32 = ['order_repeat3','cart_repeat3']
for c in f32: item_features3[c] = item_features3[c].astype('float32')
    
del tmp, tmp2
gc.collect()

113

In [9]:
item_features3.head()

,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,
18033,52,45,1,3,-1.0,1.0
18035,38,37,2,3,-1.0,1.0
18037,20,22,2,10,1.0,1.5
18044,28,24,1,3,-1.0,-1.0
18039,28,35,0,2,-1.0,2.0


In [10]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio3','count_item3','count_user3','hour_mean3','day_mean3']

In [11]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3
aid,,,,,
0,0.000000,16,14,70702.687500,389302.687500
1,0.066667,15,15,65013.533333,324213.533333
2,0.000000,3,3,60675.666667,348675.666667
3,0.116331,1341,685,62858.349739,285075.799403
4,0.050000,100,62,66462.210000,300606.210000


In [12]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std3','hour_std3','day_std3']

f32 = ['buy_ratio_std3','hour_std3','day_std3']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [13]:
item_features2.head()

,buy_ratio_std3,hour_std3,day_std3
aid,,,
836806,0.245935,14851.303711,152055.390625
479796,0.000000,1873.833008,181408.250000
67645,-1.000000,-1.000000,-1.000000
831198,0.531610,8380.233398,175784.031250
86210,0.223098,11462.041992,171459.015625


In [14]:
item_features['repeat3'] = item_features.count_item3 / item_features.count_user3

f32 = ['buy_ratio3','hour_mean3','day_mean3','repeat3']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item3','count_user3']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [15]:
item_features = cudf.concat([item_features,item_features2,item_features3],axis=1)

In [16]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,,,,,,,,,,
0,0.000000,16,14,70702.687500,389302.68750,1.142857,0.000000,7108.441895,148486.140625,12,14,0,0,-1.0,-1.000000
1,0.066667,15,15,65013.535156,324213.53125,1.000000,0.258199,9438.640625,164628.312500,14,13,0,1,-1.0,-1.000000
2,0.000000,3,3,60675.667969,348675.65625,1.000000,0.000000,8056.046875,135972.171875,2,3,0,0,-1.0,-1.000000
3,0.116331,1341,685,62858.351562,285075.81250,1.957664,0.372419,10779.861328,150338.562500,201,196,24,108,2.4,3.117647
4,0.050000,100,62,66462.210938,300606.21875,1.612903,0.219043,12213.166992,169172.562500,69,69,0,5,-1.0,1.000000


In [17]:
item_features.dtypes

buy_ratio3        float32
count_item3         int32
count_user3         int32
hour_mean3        float32
day_mean3         float32
repeat3           float32
buy_ratio_std3    float32
hour_std3         float32
day_std3          float32
prev3               int32
next3               int32
orders3             int32
carts3              int32
order_repeat3     float32
cart_repeat3      float32
dtype: object

In [18]:
COLS = [f'{x[:-1]}8' for x in item_features.columns]
item_features.columns = COLS
print(COLS)

['buy_ratio8', 'count_item8', 'count_user8', 'hour_mean8', 'day_mean8', 'repeat8', 'buy_ratio_std8', 'hour_std8', 'day_std8', 'prev8', 'next8', 'orders8', 'carts8', 'order_repeat8', 'cart_repeat8']


In [19]:
item_features.to_parquet('../../data/item_user_features/item21_LB.pqt')